# Tensorflow
tutorial for saving and restoring models  
reference: https://www.tensorflow.org/tutorials/keras/save_and_restore_models?hl=ja

In [29]:
from __future__ import absolute_import, division, print_function
import os
import tensorflow as tf
from tensorflow import keras
import pathlib

In [16]:
# datasets
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000]
test_images = test_images[:1000]

train_images = train_images.reshape(-1, 28 * 28) / 255.0
test_images = test_images.reshape(-1, 28 * 28) / 255.0

In [17]:
# define model
def create_model():
    model = keras.models.Sequential([
        keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                 loss=tf.keras.losses.sparse_categorical_crossentropy,
                 metrics=['accuracy'])
    return model

In [7]:
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [18]:
# checkpoint
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1)

model = create_model()
model.fit(train_images, train_labels, epochs=10,
         validation_data=(test_images, test_labels),
         callbacks=[cp_callback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 1s 823us/step - loss: 1.1608 - acc: 0.6760 - val_loss: 0.7506 - val_acc: 0.7740

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
1000/1000 [==============================] - 0s 349us/step - loss: 0.4364 - acc: 0.8820 - val_loss: 0.5242 - val_acc: 0.8390

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
1000/1000 [==============================] - 0s 307us/step - loss: 0.2967 - acc: 0.9220 - val_loss: 0.4895 - val_acc: 0.8490

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
1000/1000 [==============================] - 0s 272us/step - loss: 0.2176 - acc: 0.9410 - val_loss: 0.4346 - val_acc: 0.8630

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
1000/1000 [==============================] - 0s 334us/step - loss: 0.1568 - acc: 0.9690 - val_loss: 0.4291 - val_acc: 0.8670

Epoch 00005: saving model to training_1/cp.ckpt
Epoch 6/10
1000/1000 [=====

In [19]:
!ls {checkpoint_dir}

checkpoint                  cp.ckpt.index
cp.ckpt.data-00000-of-00001


In [26]:
# untrained model
un_model = create_model()
loss, acc = un_model.evaluate(test_images, test_labels)
print(loss, acc)

un_model.load_weights(checkpoint_path)
loss, acc = un_model.evaluate(test_images, test_labels)
print(loss, acc)

1000/1000 [==============================] - 0s 223us/step
2.380421806335449 0.099
1000/1000 [==============================] - 0s 42us/step
0.3909606518745422 0.871


In [28]:
# checkpoint for every 5-epochs
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
checkpoint_path, verbose=1, save_weights_only=True,
period=5)

ev_model = create_model()
ev_model.fit(train_images, train_labels,
            epochs=50,
            callbacks=[cp_callback],
            validation_data=(test_images, test_labels),
            verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [34]:
# view checkpoints
checkpoints = pathlib.Path(checkpoint_dir).glob("*.index")
checkpoints = sorted(checkpoints, key=lambda x: x.stat().st_mtime)
checkpoints = [x.with_suffix('') for x in checkpoints]
checkpoints

[PosixPath('training_2/cp-0030.ckpt'),
 PosixPath('training_2/cp-0035.ckpt'),
 PosixPath('training_2/cp-0040.ckpt'),
 PosixPath('training_2/cp-0045.ckpt'),
 PosixPath('training_2/cp-0050.ckpt')]

In [37]:
latest = str(checkpoints[-1])
re_model = create_model()
re_model.load_weights(latest)
loss, acc = re_model.evaluate(test_images, test_labels)
print(loss, acc)

1000/1000 [==============================] - 0s 138us/step
0.4760356510579586 0.881


In [40]:
# save entire model
model = create_model()
model.fit(train_images, train_labels, epochs=5)
model.save('en_model.h5')

Epoch 1/5
1000/1000 [==============================] - 1s 677us/step - loss: 1.1907 - acc: 0.6600
Epoch 2/5
1000/1000 [==============================] - 0s 242us/step - loss: 0.4345 - acc: 0.8850
Epoch 3/5
1000/1000 [==============================] - 0s 226us/step - loss: 0.2860 - acc: 0.9320
Epoch 4/5
1000/1000 [==============================] - 0s 228us/step - loss: 0.2152 - acc: 0.9440
Epoch 5/5
1000/1000 [==============================] - 0s 227us/step - loss: 0.1544 - acc: 0.9690


In [42]:
new_model = keras.models.load_model('en_model.h5')
new_model.summary()
loss, acc = new_model.evaluate(test_images, test_labels)
print(loss, acc)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_38 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_19 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
1000/1000 [==============================] - 0s 192us/step
0.4026520402431488 0.867
